In [1]:
import os
import sys
import numpy as np
import torch
import xarray as xr
from matplotlib import pyplot as plt

from scipy.linalg import pinv
import scipy
import pydmd

from tqdm import tqdm

from sklearn.decomposition import PCA

import time

import pickle as pkl

import sys
sys.path.append('../')
from utils import *

In [2]:
for scenario in [126, 245, 370, 585]:
    xr_train_emissions = load_emissions_dataset('../data/train_val/inputs_historical.nc')
    xr_test_emissions = load_emissions_dataset(f'../data/train_val/inputs_ssp{scenario}.nc')
    times, emissions_data = extract_forcing_agents(xr_train_emissions, xr_test_emissions)
    emissions_data = (emissions_data - emissions_data.mean(axis=0)) / emissions_data.std(axis=0)

    xr_train_clim = xr.open_dataset('../data/train_val/outputs_historical.nc').mean(dim='member')
    xr_test_clim = xr.open_dataset(f'../data/train_val/outputs_ssp{scenario}.nc').mean(dim='member')

    tas_all = xr.concat([xr_train_clim, xr_test_clim], dim = 'time').tas.values
    ntime, nlat, nlong = tas_all.shape
    tas_data = tas_all.reshape(ntime, -1)

    my_dmdc = pydmd.DMDc(svd_rank=5)   #(svd_rank=5) 
    my_dmdc.fit(tas_data.T, emissions_data[:-1,:].T)


    with open(f"../models/DMDc/tas{scenario}.pkl", "wb") as file:
        pkl.dump(my_dmdc, file)


    print(f'scenario {scenario} done')

scenario 126 done
scenario 245 done
scenario 370 done
scenario 585 done


In [3]:
for scenario in [126, 245, 370, 585]:
    xr_train_emissions = load_emissions_dataset('../data/train_val/inputs_historical.nc')
    xr_test_emissions = load_emissions_dataset(f'../data/train_val/inputs_ssp{scenario}.nc')
    times, emissions_data = extract_forcing_agents(xr_train_emissions, xr_test_emissions)
    emissions_data = (emissions_data - emissions_data.mean(axis=0)) / emissions_data.std(axis=0)

    xr_train_clim = xr.open_dataset('../data/train_val/outputs_historical.nc').mean(dim='member')
    xr_test_clim = xr.open_dataset(f'../data/train_val/outputs_ssp{scenario}.nc').mean(dim='member')

    pr_all = xr.concat([xr_train_clim, xr_test_clim], dim = 'time').pr.values
    ntime, nlat, nlong = tas_all.shape
    pr_data_raw = pr_all.reshape(ntime, -1)
    pr_data_less_raw =  log_transform(pr_data_raw, np.min(pr_data_raw))
    pr_data = pr_data_less_raw - np.mean(pr_data_less_raw, axis = 0)
    

    my_dmdc = pydmd.DMDc(svd_rank=5)   #(svd_rank=5) 
    my_dmdc.fit(pr_data.T, emissions_data[:-1,:].T)

    with open(f"../models/DMDc/pr{scenario}.pkl", "wb") as file:
        pkl.dump(my_dmdc, file)

    print(f'scenario {scenario} done')

scenario 126 done
scenario 245 done
scenario 370 done
scenario 585 done
